<a href="https://colab.research.google.com/github/giressesama237/Giresse-Bilal/blob/main/IoT_Based_Air_Quality_Monitoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IoT-Based Air Quality Monitoring System
## Machine Learning Model Comparison: LSTM vs Random Forest vs Linear Regression

**Dataset:** UCI Air Quality Dataset (9,358 hourly measurements)  
**Period:** March 2004 - February 2005  
**Location:** Polluted urban area, Italy  
**Objective:** Compare three ML models for air pollution forecasting

**Authors:** Giresse & Geethmal
**Date:** December 2024

In [1]:
# Cell 2: Install and Import Required Libraries

# Install additional packages if needed
!pip install -q ucimlrepo

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

All libraries imported successfully!
TensorFlow version: 2.19.0
NumPy version: 2.0.2
Pandas version: 2.2.2


In [2]:
# Cell 3: Load and Explore UCI Air Quality Dataset
from ucimlrepo import fetch_ucirepo

# Fetch dataset
air_quality = fetch_ucirepo(id=360)

# Get features and combine into single dataframe
X = air_quality.data.features
df = X.copy()

print("Dataset loaded successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nData types:")
print(df.dtypes)
print(f"\nMissing values:")
print(df.isnull().sum())

Dataset loaded successfully!

Dataset shape: (9357, 15)

First few rows:
        Date      Time  CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  \
0  3/10/2004  18:00:00     2.6         1360       150      11.9   
1  3/10/2004  19:00:00     2.0         1292       112       9.4   
2  3/10/2004  20:00:00     2.2         1402        88       9.0   
3  3/10/2004  21:00:00     2.2         1376        80       9.2   
4  3/10/2004  22:00:00     1.6         1272        51       6.5   

   PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  \
0           1046      166          1056      113          1692         1268   
1            955      103          1174       92          1559          972   
2            939      131          1140      114          1555         1074   
3            948      172          1092      122          1584         1203   
4            836      131          1205      116          1490         1110   

      T    RH      AH  
0  13.6  48.9  0.7578  
1

In [3]:
# Cell 4: Data Cleaning with Smart Missing Value Handling

# Remove empty columns (if any)
df = df.dropna(how='all', axis=1)

print("Initial dataset information:")
print(f"   Total rows: {len(df):,}")
print(f"   Total columns: {len(df.columns)}")

# Handle missing values (-200.0 indicates missing data)
print("\n Checking for missing values marked as -200.0...")

# List of numeric columns to check
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Replace -200.0 with NaN
for col in numeric_cols:
    df[col] = df[col].replace(-200.0, np.nan)

# Display missing value statistics BEFORE cleaning
print(f"\n Missing values per column:")
missing_stats = df.isnull().sum().sort_values(ascending=False)
missing_stats_pct = (missing_stats / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing_stats[missing_stats > 0],
    'Percentage': missing_stats_pct[missing_stats > 0]
})
print(missing_df)

# STRATEGY 1: Keep rows where TARGET variable is NOT missing
target_column = 'CO(GT)'

print(f"\n Strategy: Keep rows where target '{target_column}' is NOT missing")

# First, remove rows where target is missing
df_clean = df[df[target_column].notna()].copy()

print(f"\n After removing rows with missing target:")
print(f"   Remaining rows: {len(df_clean):,} ({len(df_clean)/len(df)*100:.1f}% retained)")

# STRATEGY 2: Impute missing values for FEATURES (not target)
# Define feature columns
feature_columns = [
    'PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)',
    'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH'
]

# Check which features exist
available_features = [col for col in feature_columns if col in df_clean.columns]

print(f"\n🔧 Imputing missing values for features using forward-fill + interpolation...")

# Forward fill (use previous value) then interpolate remaining gaps
for col in available_features:
    if df_clean[col].isnull().sum() > 0:
        missing_before = df_clean[col].isnull().sum()

        # Method 1: Forward fill (use last valid observation)
        df_clean[col] = df_clean[col].fillna(method='ffill')

        # Method 2: Backward fill for any remaining NaNs at start
        df_clean[col] = df_clean[col].fillna(method='bfill')

        # Method 3: Linear interpolation for any remaining gaps
        df_clean[col] = df_clean[col].interpolate(method='linear')

        missing_after = df_clean[col].isnull().sum()
        print(f"   {col}: {missing_before} → {missing_after} missing values")

# Final check: remove any rows that still have missing values (should be very few)
rows_before = len(df_clean)
df_clean = df_clean.dropna(subset=available_features + [target_column])
rows_after = len(df_clean)

if rows_before != rows_after:
    print(f"\n Removed {rows_before - rows_after} rows with remaining missing values")

print(f"\n Final cleaned dataset:")
print(f"   Total rows: {len(df_clean):,}")
print(f"   Retention rate: {len(df_clean)/len(df)*100:.1f}%")
print(f"   Rows lost: {len(df) - len(df_clean):,}")

# Create datetime column if Date and Time exist
if 'Date' in df_clean.columns and 'Time' in df_clean.columns:
    df_clean['DateTime'] = pd.to_datetime(df_clean['Date'].astype(str) + ' ' +
                                          df_clean['Time'].astype(str),
                                          errors='coerce')
    df_clean = df_clean.sort_values('DateTime')
    df_clean = df_clean.reset_index(drop=True)
    print(f"\n DateTime column created and sorted")

# Display cleaned dataset info
print(f"\nDate range: {df_clean['DateTime'].min()} to {df_clean['DateTime'].max()}")
print(f"\n Final missing values check:")
print(df_clean[available_features + [target_column]].isnull().sum())

Initial dataset information:
   Total rows: 9,357
   Total columns: 15

 Checking for missing values marked as -200.0...

 Missing values per column:
               Missing Count  Percentage
NMHC(GT)                8443       90.23
CO(GT)                  1683       17.99
NO2(GT)                 1642       17.55
NOx(GT)                 1639       17.52
PT08.S2(NMHC)            366        3.91
C6H6(GT)                 366        3.91
PT08.S1(CO)              366        3.91
PT08.S5(O3)              366        3.91
T                        366        3.91
PT08.S3(NOx)             366        3.91
PT08.S4(NO2)             366        3.91
RH                       366        3.91
AH                       366        3.91

 Strategy: Keep rows where target 'CO(GT)' is NOT missing

 After removing rows with missing target:
   Remaining rows: 7,674 (82.0% retained)

🔧 Imputing missing values for features using forward-fill + interpolation...
   PT08.S1(CO): 330 → 0 missing values
   PT08.S2(NMHC

In [4]:
# Cell 5: Feature Selection for Model Training

# Select relevant features for prediction
# We'll predict CO(GT) using sensor readings and environmental factors

feature_columns = [
    'PT08.S1(CO)',      # CO sensor
    'PT08.S2(NMHC)',    # NMHC sensor
    'PT08.S3(NOx)',     # NOx sensor
    'PT08.S4(NO2)',     # NO2 sensor
    'PT08.S5(O3)',      # O3 sensor
    'T',                # Temperature
    'RH',               # Relative Humidity
    'AH'                # Absolute Humidity
]

target_column = 'CO(GT)'  # Target: CO concentration

# Check if all columns exist
available_features = [col for col in feature_columns if col in df_clean.columns]
print(f" Available features: {len(available_features)}/{len(feature_columns)}")
print(f"Features: {available_features}")

# Create feature matrix and target vector
X_data = df_clean[available_features].values
y_data = df_clean[target_column].values

print(f"\n Feature matrix shape: {X_data.shape}")
print(f" Target vector shape: {y_data.shape}")

# Display basic statistics
print(f"\n Target variable (CO) statistics:")
print(f"Mean: {y_data.mean():.2f} mg/m³")
print(f"Std: {y_data.std():.2f} mg/m³")
print(f"Min: {y_data.min():.2f} mg/m³")
print(f"Max: {y_data.max():.2f} mg/m³")

 Available features: 8/8
Features: ['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']

 Feature matrix shape: (7674, 8)
 Target vector shape: (7674,)

 Target variable (CO) statistics:
Mean: 2.15 mg/m³
Std: 1.45 mg/m³
Min: 0.10 mg/m³
Max: 11.90 mg/m³


In [5]:
# Cell 6: Temporal Train/Validation/Test Split
# IMPORTANT: No random shuffling for time-series data!

# Split ratios
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

# Calculate split indices
n_samples = len(X_data)
train_end = int(n_samples * train_ratio)
val_end = int(n_samples * (train_ratio + val_ratio))

# Perform temporal split
X_train = X_data[:train_end]
y_train = y_data[:train_end]

X_val = X_data[train_end:val_end]
y_val = y_data[train_end:val_end]

X_test = X_data[val_end:]
y_test = y_data[val_end:]

print(" Temporal split completed!")
print(f"\n Training set: {len(X_train)} samples ({train_ratio*100:.0f}%)")
print(f" Validation set: {len(X_val)} samples ({val_ratio*100:.0f}%)")
print(f" Test set: {len(X_test)} samples ({test_ratio*100:.0f}%)")

# Display date ranges if DateTime exists
if 'DateTime' in df_clean.columns:
    print(f"\n Training period: {df_clean['DateTime'].iloc[0]} to {df_clean['DateTime'].iloc[train_end-1]}")
    print(f" Validation period: {df_clean['DateTime'].iloc[train_end]} to {df_clean['DateTime'].iloc[val_end-1]}")
    print(f" Test period: {df_clean['DateTime'].iloc[val_end]} to {df_clean['DateTime'].iloc[-1]}")

 Temporal split completed!

 Training set: 5371 samples (70%)
 Validation set: 1151 samples (15%)
 Test set: 1152 samples (15%)

 Training period: 2004-03-10 18:00:00 to 2004-12-21 11:00:00
 Validation period: 2004-12-21 12:00:00 to 2005-02-14 11:00:00
 Test period: 2005-02-14 12:00:00 to 2005-04-04 14:00:00


In [6]:
# Cell 7: Data Normalization (Min-Max Scaling)
# CRITICAL: Fit scaler only on training data to prevent data leakage!

# Initialize scaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit on training data only
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

# Transform validation and test data using training parameters
X_val_scaled = scaler_X.transform(X_val)
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1)).flatten()

X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

print(" Data normalization completed!")
print(f"\n Scaled feature range: [{X_train_scaled.min():.3f}, {X_train_scaled.max():.3f}]")
print(f" Scaled target range: [{y_train_scaled.min():.3f}, {y_train_scaled.max():.3f}]")

# Verify shapes
print(f"\n Training shapes:")
print(f"   X_train_scaled: {X_train_scaled.shape}")
print(f"   y_train_scaled: {y_train_scaled.shape}")

 Data normalization completed!

 Scaled feature range: [0.000, 1.000]
 Scaled target range: [0.000, 1.000]

 Training shapes:
   X_train_scaled: (5371, 8)
   y_train_scaled: (5371,)


In [7]:
# Cell 8: Create Temporal Sequences for LSTM Model

def create_sequences(X, y, seq_length=24):
    """
    Create temporal sequences for LSTM training

    Args:
        X: Feature array (samples, features)
        y: Target array (samples,)
        seq_length: Number of time steps to look back (default: 24 hours)

    Returns:
        X_seq: Sequences (samples, timesteps, features)
        y_seq: Targets (samples,)
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        # Input: previous seq_length time steps
        X_seq.append(X[i:i+seq_length])
        # Output: value at current time step
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# Create sequences with 24-hour lookback window
SEQUENCE_LENGTH = 24

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, SEQUENCE_LENGTH)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, SEQUENCE_LENGTH)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, SEQUENCE_LENGTH)

print(" Sequences created for LSTM!")
print(f"\n Sequence shapes:")
print(f"   X_train_seq: {X_train_seq.shape} (samples, timesteps, features)")
print(f"   y_train_seq: {y_train_seq.shape}")
print(f"   X_val_seq: {X_val_seq.shape}")
print(f"   y_val_seq: {y_val_seq.shape}")
print(f"   X_test_seq: {X_test_seq.shape}")
print(f"   y_test_seq: {y_test_seq.shape}")

print(f"\n Each sample uses {SEQUENCE_LENGTH} previous hours to predict the next hour")

 Sequences created for LSTM!

 Sequence shapes:
   X_train_seq: (5347, 24, 8) (samples, timesteps, features)
   y_train_seq: (5347,)
   X_val_seq: (1127, 24, 8)
   y_val_seq: (1127,)
   X_test_seq: (1128, 24, 8)
   y_test_seq: (1128,)

 Each sample uses 24 previous hours to predict the next hour


In [8]:
# Cell 9: Create Temporal Sequences for LSTM Model

def create_sequences(X, y, seq_length=24):
    """
    Create temporal sequences for LSTM training

    Args:
        X: Feature array (samples, features)
        y: Target array (samples,)
        seq_length: Number of time steps to look back (default: 24 hours)

    Returns:
        X_seq: Sequences (samples, timesteps, features)
        y_seq: Targets (samples,)
    """
    X_seq, y_seq = [], []

    for i in range(len(X) - seq_length):
        # Input: previous seq_length time steps
        X_seq.append(X[i:i+seq_length])
        # Output: value at current time step
        y_seq.append(y[i+seq_length])

    return np.array(X_seq), np.array(y_seq)

# Create sequences with 24-hour lookback window
SEQUENCE_LENGTH = 24

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, SEQUENCE_LENGTH)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, SEQUENCE_LENGTH)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, SEQUENCE_LENGTH)

print(" Sequences created for LSTM!")
print(f"\n Sequence shapes:")
print(f"   X_train_seq: {X_train_seq.shape} (samples, timesteps, features)")
print(f"   y_train_seq: {y_train_seq.shape}")
print(f"   X_val_seq: {X_val_seq.shape}")
print(f"   y_val_seq: {y_val_seq.shape}")
print(f"   X_test_seq: {X_test_seq.shape}")
print(f"   y_test_seq: {y_test_seq.shape}")

print(f"\n Each sample uses {SEQUENCE_LENGTH} previous hours to predict the next hour")

 Sequences created for LSTM!

 Sequence shapes:
   X_train_seq: (5347, 24, 8) (samples, timesteps, features)
   y_train_seq: (5347,)
   X_val_seq: (1127, 24, 8)
   y_val_seq: (1127,)
   X_test_seq: (1128, 24, 8)
   y_test_seq: (1128,)

 Each sample uses 24 previous hours to predict the next hour


In [9]:
# Cell 10: Build LSTM Neural Network Model

def build_lstm_model(input_shape):
    """
    Build 3-layer LSTM architecture

    Args:
        input_shape: Tuple (timesteps, features)

    Returns:
        Compiled LSTM model
    """
    model = Sequential([
        # First LSTM layer: 128 units
        LSTM(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer: 64 units
        LSTM(64, return_sequences=True),
        Dropout(0.2),

        # Third LSTM layer: 32 units
        LSTM(32, return_sequences=False),
        Dropout(0.2),

        # Dense layers
        Dense(16, activation='relu'),
        Dense(1)  # Output layer
    ])

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    return model

# Build model
lstm_model = build_lstm_model(input_shape=(SEQUENCE_LENGTH, X_train_scaled.shape[1]))

# Display model architecture
print("LSTM Model Architecture:")
lstm_model.summary()

# Count parameters
total_params = lstm_model.count_params()
print(f"\n Total parameters: {total_params:,}")

LSTM Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 24, 128)        │        70,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 24, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 132,513 (517.63 KB)

 Trainable params: 132,513 (517.63 KB)

 Non-trainable params: 0 (0.00 B)


 Total parameters: 132,513


In [ ]:
# Cell 11: Train LSTM Model

# Define callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

# Train model
print("🚀 Training LSTM model...")
print("This may take 10-20 minutes depending on your hardware\n")

history = lstm_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\nLSTM training completed!")

🚀 Training LSTM model...
This may take 10-20 minutes depending on your hardware

Epoch 1/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 32s 112ms/step - loss: 0.0160 - mae: 0.0965 - val_loss: 0.0157 - val_mae: 0.0971 - learning_rate: 0.0010
Epoch 2/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - loss: 0.0120 - mae: 0.0806 - val_loss: 0.0135 - val_mae: 0.0885 - learning_rate: 0.0010
Epoch 3/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - loss: 0.0105 - mae: 0.0760 - val_loss: 0.0136 - val_mae: 0.0896 - learning_rate: 0.0010
Epoch 4/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - loss: 0.0091 - mae: 0.0702 - val_loss: 0.0130 - val_mae: 0.0871 - learning_rate: 0.0010
Epoch 5/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - loss: 0.0086 - mae: 0.0682 - val_loss: 0.0119 - val_mae: 0.0818 - learning_rate: 0.0010
Epoch 6/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step - loss: 0.0079 - mae: 0.0648 - val_loss: 0.0105 - val_mae: 0.0770 - learning_rate: 0.0010
Epoch 7/50
168/168 ━━━━━━━━━━━━━━━━━━━━ 11s 64ms/step -

In [ ]:
# Cell 12: Visualize LSTM Training History

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_title('LSTM Model Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Mean Squared Error')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot MAE
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_title('LSTM Model MAE', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Mean Absolute Error')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Best epoch: {np.argmin(history.history['val_loss']) + 1}")
print(f"Best validation loss: {min(history.history['val_loss']):.6f}")

In [ ]:
# Cell 13: Build and Train Random Forest Model

print("Training Random Forest model...")

# Initialize Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,        # Number of trees
    max_depth=10,            # Maximum tree depth
    min_samples_split=5,     # Minimum samples to split
    min_samples_leaf=2,      # Minimum samples in leaf
    bootstrap=True,          # Bootstrap sampling
    random_state=42,
    n_jobs=-1,               # Use all CPU cores
    verbose=1
)

# Train model (no sequences needed - uses tabular data)
rf_model.fit(X_train_scaled, y_train_scaled)

print("\n Random Forest training completed!")

# Display feature importance
feature_importance = pd.DataFrame({
    'Feature': available_features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n Feature Importance Ranking:")
print(feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.xlabel('Importance Score')
plt.title('Random Forest Feature Importance', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Cell 14: Build and Train Ridge Linear Regression with Polynomial Features

print("Training Linear Regression model...")

# Create polynomial features (degree 2 for non-linearity)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform features
X_train_poly = poly.fit_transform(X_train_scaled)
X_val_poly = poly.transform(X_val_scaled)
X_test_poly = poly.transform(X_test_scaled)

print(f"Polynomial features created: {X_train_poly.shape[1]} features")

# Train Ridge Regression (with regularization)
lr_model = Ridge(alpha=1.0, random_state=42)
lr_model.fit(X_train_poly, y_train_scaled)

print("Linear Regression training completed!")

# Display model coefficients
print(f"\n Model statistics:")
print(f"   Number of coefficients: {len(lr_model.coef_)}")
print(f"   Intercept: {lr_model.intercept_:.6f}")
print(f"   Regularization (alpha): {lr_model.alpha}")

In [ ]:
# Cell 15: Generate Predictions on Test Set (ALIGNED)

print(" Generating predictions on test set...\n")

# IMPORTANT: All models must predict on the SAME samples for fair comparison
# LSTM uses sequences, so we align RF and LR to match

# Step 1: LSTM predictions (uses sequences)
print("LSTM predictions...")
lstm_pred_scaled = lstm_model.predict(X_test_seq, verbose=0)
# Inverse transform to original scale
lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled).flatten()
# Corresponding true values
y_test_lstm = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

print(f"   LSTM predictions shape: {lstm_pred.shape}")
print(f"   LSTM can predict on {len(lstm_pred)} samples (due to sequence requirement)")

# Step 2: Align Random Forest and Linear Regression
# Skip the first SEQUENCE_LENGTH samples to align with LSTM
print(f"\n Aligning Random Forest and Linear Regression...")
print(f"   Skipping first {SEQUENCE_LENGTH} samples to match LSTM sequences")

# RF predictions - aligned with LSTM
# We need to skip first SEQUENCE_LENGTH samples from test set
X_test_scaled_aligned = X_test_scaled[SEQUENCE_LENGTH:SEQUENCE_LENGTH + len(lstm_pred)]
y_test_aligned = y_test[SEQUENCE_LENGTH:SEQUENCE_LENGTH + len(lstm_pred)]

# Random Forest predictions
rf_pred_scaled = rf_model.predict(X_test_scaled_aligned)
rf_pred = scaler_y.inverse_transform(rf_pred_scaled.reshape(-1, 1)).flatten()

print(f"   Random Forest predictions shape: {rf_pred.shape}")

# Step 3: Linear Regression predictions (polynomial features)
X_test_poly_aligned = X_test_poly[SEQUENCE_LENGTH:SEQUENCE_LENGTH + len(lstm_pred)]
lr_pred_scaled = lr_model.predict(X_test_poly_aligned)
lr_pred = scaler_y.inverse_transform(lr_pred_scaled.reshape(-1, 1)).flatten()

print(f"   Linear Regression predictions shape: {lr_pred.shape}")

# Verify all predictions have same length
print(f"\n Alignment verification:")
print(f"   True values: {len(y_test_aligned)} samples")
print(f"   LSTM predictions: {len(lstm_pred)} samples")
print(f"   RF predictions: {len(rf_pred)} samples")
print(f"   LR predictions: {len(lr_pred)} samples")

# Check if all have same length
if len(y_test_aligned) == len(lstm_pred) == len(rf_pred) == len(lr_pred):
    print(f"\n All predictions ALIGNED correctly!")
else:
    print(f"\n WARNING: Predictions have different lengths!")
    print(f"   Truncating to minimum length...")
    min_len = min(len(y_test_aligned), len(lstm_pred), len(rf_pred), len(lr_pred))
    y_test_aligned = y_test_aligned[:min_len]
    lstm_pred = lstm_pred[:min_len]
    rf_pred = rf_pred[:min_len]
    lr_pred = lr_pred[:min_len]
    print(f"   Final length: {min_len} samples")

In [ ]:
# Cell 15.5: DEBUGGING LSTM Predictions

print("DEBUGGING LSTM Predictions...")
print("=" * 80)

# 1. Check shapes
print("\n Shapes Check:")
print(f"   lstm_pred_scaled shape: {lstm_pred_scaled.shape}")
print(f"   lstm_pred (after inverse) shape: {lstm_pred.shape}")
print(f"   y_test_aligned shape: {y_test_aligned.shape}")

# 2. Check value ranges
print("\n Value Ranges Check:")
print(f"   lstm_pred_scaled range: [{lstm_pred_scaled.min():.4f}, {lstm_pred_scaled.max():.4f}]")
print(f"   lstm_pred range: [{lstm_pred.min():.4f}, {lstm_pred.max():.4f}]")
print(f"   y_test_aligned range: [{y_test_aligned.min():.4f}, {y_test_aligned.max():.4f}]")

# 3. Check first 10 predictions vs true values
print("\n First 10 Predictions vs True Values:")
print(f"{'Index':<10} {'True Value':<15} {'LSTM Pred':<15} {'Error':<15}")
print("-" * 60)
for i in range(min(10, len(y_test_aligned))):
    error = abs(y_test_aligned[i] - lstm_pred[i])
    print(f"{i:<10} {y_test_aligned[i]:<15.4f} {lstm_pred[i]:<15.4f} {error:<15.4f}")

# 4. Check if predictions are constant (model collapsed)
lstm_std = np.std(lstm_pred)
print(f"\n Prediction Variability:")
print(f"   LSTM predictions std dev: {lstm_std:.6f}")
if lstm_std < 0.01:
    print("    WARNING: LSTM predictions are nearly constant!")
else:
    print("    LSTM predictions vary normally")

# 5. Visualize first 100 predictions
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
n_vis = min(100, len(y_test_aligned))
plt.plot(range(n_vis), y_test_aligned[:n_vis], label='True Values', linewidth=2, color='black')
plt.plot(range(n_vis), lstm_pred[:n_vis], label='LSTM Predictions', linewidth=1.5, color='blue', alpha=0.7)
plt.xlabel('Sample Index')
plt.ylabel('CO Concentration')
plt.title('LSTM Predictions vs True Values (First 100 Samples)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 6. Check for NaN or Inf
print(f"\n Data Quality Check:")
print(f"   LSTM predictions - NaN count: {np.isnan(lstm_pred).sum()}")
print(f"   LSTM predictions - Inf count: {np.isinf(lstm_pred).sum()}")
print(f"   True values - NaN count: {np.isnan(y_test_aligned).sum()}")

print("\n" + "=" * 80)


In [ ]:
# Cell 16: Calculate Performance Metrics for All Models (ALIGNED)

def calculate_metrics(y_true, y_pred, model_name):
    """
    Calculate regression metrics

    Args:
        y_true: True values
        y_pred: Predicted values
        model_name: Name of the model

    Returns:
        Dictionary of metrics
    """
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)

    # Calculate MAPE (Mean Absolute Percentage Error)
    # Avoid division by zero
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

    return {
        'Model': model_name,
        'R²': r2,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE (%)': mape
    }

# Calculate metrics for each model using ALIGNED predictions
print(" Calculating performance metrics on ALIGNED test set...\n")

# All models use the SAME y_test_aligned for fair comparison
lstm_metrics = calculate_metrics(y_test_aligned, lstm_pred, 'LSTM')
rf_metrics = calculate_metrics(y_test_aligned, rf_pred, 'Random Forest')
lr_metrics = calculate_metrics(y_test_aligned, lr_pred, 'Linear Regression')

# Create comparison dataframe
results_df = pd.DataFrame([lstm_metrics, rf_metrics, lr_metrics])

print(" Performance Metrics on Test Set (ALIGNED):")
print("=" * 80)
print(results_df.to_string(index=False))
print("=" * 80)

# Verify R² values are reasonable
print("\n Sanity check:")
for _, row in results_df.iterrows():
    if row['R²'] < 0 or row['R²'] > 1:
        print(f"    {row['Model']}: R² = {row['R²']:.4f} (UNUSUAL!)")
    else:
        print(f"    {row['Model']}: R² = {row['R²']:.4f} (OK)")

# Highlight best model for each metric
print("\n Best performing model per metric:")
print(f"   Highest R²: {results_df.loc[results_df['R²'].idxmax(), 'Model']} ({results_df['R²'].max():.4f})")
print(f"   Lowest RMSE: {results_df.loc[results_df['RMSE'].idxmin(), 'Model']} ({results_df['RMSE'].min():.4f})")
print(f"   Lowest MAE: {results_df.loc[results_df['MAE'].idxmin(), 'Model']} ({results_df['MAE'].min():.4f})")
print(f"   Lowest MAPE: {results_df.loc[results_df['MAPE (%)'].idxmin(), 'Model']} ({results_df['MAPE (%)'].min():.2f}%)")

In [ ]:
# Cell 17: Weighted Ensemble (Optimized Weights)

print(" Creating weighted ensemble model...\n")

# Poids basés sur performance:
# RF (meilleur) = 50%
# LR (2ème) = 30%
# LSTM (3ème) = 20%

ensemble_pred = (0.5 * rf_pred +
                 0.3 * lr_pred +
                 0.2 * lstm_pred)

print(f" Ensemble predictions shape: {ensemble_pred.shape}")

# Calculate ensemble metrics
ensemble_metrics = calculate_metrics(y_test_aligned, ensemble_pred, 'Ensemble (0.5RF+0.3LR+0.2LSTM)')

# Add to results
results_df = pd.concat([results_df, pd.DataFrame([ensemble_metrics])], ignore_index=True)

print("\n Final Model Comparison (including Ensemble):")
print("=" * 85)
print(results_df.to_string(index=False))
print("=" * 85)

# Compare ensemble vs best individual
best_individual_r2 = results_df[results_df['Model'] != 'Ensemble (0.5RF+0.3LR+0.2LSTM)']['R²'].max()
ensemble_r2 = ensemble_metrics['R²']
improvement = ((ensemble_r2 - best_individual_r2) / best_individual_r2) * 100

if improvement > 0:
    print(f"\n Ensemble improvement: +{improvement:.2f}%")
    print(f" Ensemble R² = {ensemble_r2:.4f} vs Best Individual R² = {best_individual_r2:.4f}")
else:
    print(f"\n Ensemble R² = {ensemble_r2:.4f} (comparable to best individual)")

In [ ]:
# Cell 18: Visualize Model Predictions (SEPARATE FIGURES)

import matplotlib.pyplot as plt
import numpy as np

def plot_timeseries_single(y_true, y_pred, model_name, r2, color='blue', n_samples=500, filename=None):
    """
    Create individual time-series prediction plot for one model

    Args:
        y_true: True CO values
        y_pred: Predicted CO values
        model_name: Name of the model
        r2: R² score
        color: Color for predictions line
        n_samples: Number of samples to plot
        filename: Path to save figure
    """
    n_samples = min(n_samples, len(y_true))
    x_axis = range(n_samples)

    fig, ax = plt.subplots(figsize=(10, 6))  # Optimized size for LaTeX

    # Plot true values (black line)
    ax.plot(x_axis, y_true[:n_samples], label='True Values',
            linewidth=2, alpha=0.8, color='black')

    # Plot predictions (colored line)
    ax.plot(x_axis, y_pred[:n_samples], label=f'{model_name} Predictions',
            linewidth=1.5, alpha=0.7, color=color)

    ax.set_xlabel('Time Steps', fontsize=12)
    ax.set_ylabel('CO Concentration (mg/m³)', fontsize=12)
    ax.set_title(f'{model_name} Model\nR² = {r2:.4f}', fontsize=13, fontweight='bold')
    ax.legend(loc='upper right', fontsize=11)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()

    # Save if filename provided
    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"    Saved: {filename}")

    plt.show()
    plt.close()

# ============================================================================
# Generate 4 Separate Time-Series Figures
# ============================================================================

print(" Generating 4 separate time-series prediction figures...\n")

# Figure 1: LSTM
print("LSTM Time-Series Plot...")
plot_timeseries_single(
    y_test_aligned,
    lstm_pred,
    'LSTM',
    lstm_metrics['R²'],
    color='blue',
    n_samples=500,
    filename='fig_timeseries_lstm.png'
)

# Figure 2: Random Forest
print("Random Forest Time-Series Plot...")
plot_timeseries_single(
    y_test_aligned,
    rf_pred,
    'Random Forest',
    rf_metrics['R²'],
    color='green',
    n_samples=500,
    filename='fig_timeseries_rf.png'
)

# Figure 3: Linear Regression
print("Linear Regression Time-Series Plot...")
plot_timeseries_single(
    y_test_aligned,
    lr_pred,
    'Linear Regression',
    lr_metrics['R²'],
    color='red',
    n_samples=500,
    filename='fig_timeseries_lr.png'
)

# Figure 4: Ensemble
print(" Ensemble Time-Series Plot...")
plot_timeseries_single(
    y_test_aligned,
    ensemble_pred,
    'Ensemble',
    ensemble_metrics['R²'],
    color='purple',
    n_samples=500,
    filename='fig_timeseries_ensemble.png'
)

print("\n" + "="*70)
print("ALL 4 TIME-SERIES PLOTS GENERATED AND SAVED!")
print("="*70)
print("\nFiles saved:")
print("  1. fig_timeseries_lstm.png")
print("  2. fig_timeseries_rf.png")
print("  3. fig_timeseries_lr.png")
print("  4. fig_timeseries_ensemble.png")
print("\nYou can now right-click and save each image separately!")
print("="*70)

In [ ]:
# Cell 19: Scatter Plots - True vs Predicted Values (SEPARATE FIGURES)

import matplotlib.pyplot as plt
import numpy as np

# Function to plot scatter with regression line and legend
def plot_scatter_single(y_true, y_pred, model_name, r2, filename=None):
    """
    Create individual scatter plot for one model
    """
    fig, ax = plt.subplots(figsize=(8, 8))

    # Scatter plot with label
    ax.scatter(y_true, y_pred, alpha=0.5, s=15, label='Predictions')

    # Perfect prediction line (y=x)
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')

    ax.set_xlabel('True CO Concentration (mg/m³)', fontsize=13)
    ax.set_ylabel('Predicted CO Concentration (mg/m³)', fontsize=13)
    ax.set_title(f'{model_name} Model\nR² = {r2:.4f}', fontsize=14, fontweight='bold')
    ax.legend(loc='upper left', fontsize=11)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()

    # Save if filename provided
    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"   Saved: {filename}")

    plt.show()
    plt.close()

# ============================================================================
# Generate 4 Separate Figures
# ============================================================================

print("Generating 4 separate scatter plot figures...\n")

# Figure 1: LSTM
print("LSTM Scatter Plot...")
plot_scatter_single(
    y_test_aligned,
    lstm_pred,
    'LSTM',
    lstm_metrics['R²'],
    filename='fig_scatter_lstm.png'
)

# Figure 2: Random Forest
print("Random Forest Scatter Plot...")
plot_scatter_single(
    y_test_aligned,
    rf_pred,
    'Random Forest',
    rf_metrics['R²'],
    filename='fig_scatter_rf.png'
)

# Figure 3: Linear Regression
print("Linear Regression Scatter Plot...")
plot_scatter_single(
    y_test_aligned,
    lr_pred,
    'Linear Regression',
    lr_metrics['R²'],
    filename='fig_scatter_lr.png'
)

# Figure 4: Ensemble
print("Ensemble Scatter Plot...")
plot_scatter_single(
    y_test_aligned,
    ensemble_pred,
    'Ensemble',
    ensemble_metrics['R²'],
    filename='fig_scatter_ensemble.png'
)

print("\n" + "="*70)
print("ALL 4 SCATTER PLOTS GENERATED AND SAVED!")
print("="*70)
print("\nFiles saved:")
print("  1. fig_scatter_lstm.png")
print("  2. fig_scatter_rf.png")
print("  3. fig_scatter_lr.png")
print("  4. fig_scatter_ensemble.png")
print("\nYou can now right-click and save each image separately!")
print("="*70)

In [ ]:
# Cell 20: Error Analysis and Distribution (SEPARATE FIGURES)

import matplotlib.pyplot as plt
import numpy as np

# Calculate residuals (errors)
lstm_residuals = y_test_aligned - lstm_pred
rf_residuals = y_test_aligned - rf_pred
lr_residuals = y_test_aligned - lr_pred
ensemble_residuals = y_test_aligned - ensemble_pred

def plot_residuals_single(residuals, model_name, color='blue', filename=None):
    """
    Create individual residual distribution plot for one model

    Args:
        residuals: Array of residual errors
        model_name: Name of the model
        color: Color for histogram
        filename: Path to save figure
    """
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot histogram
    ax.hist(residuals, bins=50, alpha=0.7, color=color, edgecolor='black')

    # Add vertical line at zero (perfect prediction)
    ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')

    # Calculate statistics
    mean_error = residuals.mean()
    std_error = residuals.std()
    max_error = np.abs(residuals).max()

    # Add statistics text box
    stats_text = f'Mean: {mean_error:.3f} mg/m³\nStd Dev: {std_error:.3f} mg/m³\nMax Error: {max_error:.3f} mg/m³'
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
            fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    ax.set_xlabel('Residuals (mg/m³)', fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title(f'{model_name} Residuals Distribution', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()

    # Save if filename provided
    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f" Saved: {filename}")

    plt.show()
    plt.close()

# ============================================================================
# Generate 4 Separate Residual Distribution Figures
# ============================================================================

print("Generating 4 separate residual distribution figures...\n")

# Figure 1: LSTM
print("LSTM Residuals...")
plot_residuals_single(
    lstm_residuals,
    'LSTM',
    color='blue',
    filename='fig_residuals_lstm.png'
)

# Figure 2: Random Forest
print("Random Forest Residuals...")
plot_residuals_single(
    rf_residuals,
    'Random Forest',
    color='green',
    filename='fig_residuals_rf.png'
)

# Figure 3: Linear Regression
print("Linear Regression Residuals...")
plot_residuals_single(
    lr_residuals,
    'Linear Regression',
    color='red',
    filename='fig_residuals_lr.png'
)

# Figure 4: Ensemble
print(" Ensemble Residuals...")
plot_residuals_single(
    ensemble_residuals,
    'Ensemble',
    color='purple',
    filename='fig_residuals_ensemble.png'
)

# ============================================================================
# Print Comprehensive Residual Statistics
# ============================================================================

print("\n" + "="*70)
print(" RESIDUAL STATISTICS SUMMARY")
print("="*70)
print(f"{'Model':<20} {'Mean Error':<15} {'Std Dev':<15} {'Max Error':<15}")
print("="*70)
print(f"{'LSTM':<20} {lstm_residuals.mean():<15.4f} {lstm_residuals.std():<15.4f} {np.abs(lstm_residuals).max():<15.4f}")
print(f"{'Random Forest':<20} {rf_residuals.mean():<15.4f} {rf_residuals.std():<15.4f} {np.abs(rf_residuals).max():<15.4f}")
print(f"{'Linear Regression':<20} {lr_residuals.mean():<15.4f} {lr_residuals.std():<15.4f} {np.abs(lr_residuals).max():<15.4f}")
print(f"{'Ensemble':<20} {ensemble_residuals.mean():<15.4f} {ensemble_residuals.std():<15.4f} {np.abs(ensemble_residuals).max():<15.4f}")
print("="*70)

print("\n Interpretation:")
print("• Mean Error near 0 = Unbiased predictions")
print("• Low Std Dev = Consistent accuracy")
print("• Symmetric distribution around 0 = No systematic bias")

print("\n" + "="*70)
print("ALL 4 RESIDUAL PLOTS GENERATED AND SAVED!")
print("="*70)
print("\nFiles saved:")
print("  1. fig_residuals_lstm.png")
print("  2. fig_residuals_rf.png")
print("  3. fig_residuals_lr.png")
print("  4. fig_residuals_ensemble.png")
print("="*70)

In [ ]:
# Cell 21: Bar Chart Comparison of Model Performance

# Prepare data for visualization
models = results_df['Model'].values
r2_scores = results_df['R²'].values
rmse_scores = results_df['RMSE'].values
mae_scores = results_df['MAE'].values

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Bar colors
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']

# Plot R² scores
axes[0].bar(range(len(models)), r2_scores, color=colors)
axes[0].set_xticks(range(len(models)))
axes[0].set_xticklabels(models, rotation=45, ha='right')
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('R² Score Comparison\n(Higher is Better)', fontsize=13, fontweight='bold')
axes[0].set_ylim([0.8, 1.0])
axes[0].grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for i, v in enumerate(r2_scores):
    axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

# Plot RMSE
axes[1].bar(range(len(models)), rmse_scores, color=colors)
axes[1].set_xticks(range(len(models)))
axes[1].set_xticklabels(models, rotation=45, ha='right')
axes[1].set_ylabel('RMSE (mg/m³)', fontsize=12)
axes[1].set_title('RMSE Comparison\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for i, v in enumerate(rmse_scores):
    axes[1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

# Plot MAE
axes[2].bar(range(len(models)), mae_scores, color=colors)
axes[2].set_xticks(range(len(models)))
axes[2].set_xticklabels(models, rotation=45, ha='right')
axes[2].set_ylabel('MAE (mg/m³)', fontsize=12)
axes[2].set_title('MAE Comparison\n(Lower is Better)', fontsize=13, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')
# Add value labels on bars
for i, v in enumerate(mae_scores):
    axes[2].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("Bar chart comparison completed!")

# Task
Load and parse the "Air Quality Standards Table.csv" file to extract AQI breakpoints for CO, C6H6, NO2, NOx, and O3, storing them in a structured format for easy lookup. Then, prepare the pollutant data by extracting predicted CO values from `ensemble_pred` and actual C6H6(GT), NO2(GT), NOx(GT), and PT08.S5(O3) values from `df_clean`, ensuring all data is aligned with `y_test_aligned` for subsequent AQI calculations.

**Reasoning**:
First, I will load the 'Air Quality Standards Table.csv' file into a pandas DataFrame called `aqi_standards_df` and then display its head, info, and columns to understand its structure.



## Load and Parse AQI Standards

### Subtask:
Load the 'Air Quality Standards Table.csv' and parse it into a structured format (e.g., dictionary or DataFrame) that allows easy lookup of AQI breakpoints for each specified pollutant (CO, C6H6, NO2, NOx, O3).


# Task
# Plan:
* **Add pandas import to the cell**: To make the selected code runnable in isolation, I will add `import pandas as pd` to the beginning of the cell. This ensures the `pd` alias is recognized when `pd.read_csv` is called.
* **Execute the modified cell**: After adding the import statement, the cell will be executed to load the AQI standards table and display its initial information.
* **Proceed with AQI parsing**: Once the DataFrame is successfully loaded, the next steps will involve parsing this DataFrame to extract the necessary AQI breakpoints for each pollutant.

Load the 'Air Quality Standards Table.csv' file into a pandas DataFrame and display its head, info, and columns.

## Load and Parse AQI Standards

### Subtask:
Load the 'Air Quality Standards Table.csv' file into a pandas DataFrame and display its head, info, and columns.


**Reasoning**:
Load the 'Air Quality Standards Table.csv' file into a pandas DataFrame and display its head, info, and columns as requested in the subtask.



In [ ]:
import pandas as pd

aqi_standards_df = pd.read_csv('/content/Air Quality Standards Table.csv', encoding='latin-1')

print("First 5 rows of AQI Standards Table:")
print(aqi_standards_df.head())
print("\nDataFrame Info:")
aqi_standards_df.info()
print("\nColumn Names:")
print(aqi_standards_df.columns.tolist())

# Display the full table with all rows and columns visible
print("\n" + "="*80)
print("FULL AQI STANDARDS TABLE:")
print("="*80)
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect width
pd.set_option('display.max_colwidth', None)  # Show full column content
display(aqi_standards_df)  # Use display() for better formatting in notebooks

**Reasoning**:
The previous output showed that the AQI standards table needs cleaning and reformatting to be useful. The first row contains the actual column names (pollutant names), and the data starts from the second row. I will set the first row as the new header, clean up the column names, and then parse the AQI ranges and pollutant concentration ranges into a structured format for each pollutant.



# Task
Clean the `aqi_standards_df` by setting the first row as the header, renaming columns, and then parsing the AQI and pollutant concentration ranges for CO, C6H6, NO2, NOx, and O3. Store these parsed breakpoints in a structured dictionary named `aqi_breakpoints` for easy lookup. Finally, verify the structure of `aqi_breakpoints`.

## Clean the aqi_standards_df and parse AQI breakpoints

### Subtask:
Clean the `aqi_standards_df` by setting the first row as the header, renaming columns, and parsing AQI and pollutant concentration ranges for CO, C6H6, NO2, NOx, and O3 into a structured dictionary `aqi_breakpoints`.


**Reasoning**:
The subtask requires cleaning the AQI standards DataFrame, setting the correct header, renaming columns, and parsing the AQI and pollutant concentration ranges into a structured dictionary. This code block will perform all these steps by defining helper functions for parsing ranges and then iterating through the cleaned DataFrame to populate the `aqi_breakpoints` dictionary as specified.



In [ ]:
import numpy as np
import re

# 1. Create a copy of the aqi_standards_df and name it aqi_df_cleaned.
aqi_df_cleaned = aqi_standards_df.copy()

# 2. Set the first row (index 0) of aqi_df_cleaned as the new column headers.
aqi_df_cleaned.columns = aqi_df_cleaned.iloc[0]

# 3. Remove the first row from aqi_df_cleaned and reset the index.
aqi_df_cleaned = aqi_df_cleaned[1:].reset_index(drop=True)

# Print columns to see what we have
print("=" * 80)
print("ACTUAL COLUMNS IN THE DATAFRAME:")
print("=" * 80)
for i, col in enumerate(aqi_df_cleaned.columns):
    print(f"{i}: '{col}' (type: {type(col).__name__})")
print("\n")

# Display the dataframe to see the data
print("First few rows:")
print(aqi_df_cleaned.head())
print("\n")

# 4. Create a more flexible renaming map by searching for keywords
new_column_names = {}
for col in aqi_df_cleaned.columns:
    col_str = str(col).strip()

    # Check for each column type
    if pd.isna(col) or col_str == 'nan' or col_str == '':
        new_column_names[col] = 'AQI Category'
    elif 'AQI' in col_str and ('Range' in col_str or 'Values' in col_str):
        new_column_names[col] = 'AQI Range'
    elif 'CO' in col_str and 'Carbon' in col_str:
        new_column_names[col] = 'CO'
    elif 'C6H6' in col_str or 'Benzene' in col_str:
        new_column_names[col] = 'C6H6'
    elif 'NO2' in col_str and 'Nitrogen Dioxide' in col_str:
        new_column_names[col] = 'NO2'
    elif 'NOx' in col_str or 'Nitrogen Oxides' in col_str:
        new_column_names[col] = 'NOx'
    elif 'O3' in col_str or 'Ozone' in col_str:
        new_column_names[col] = 'O3'
    elif 'Health' in col_str:
        new_column_names[col] = 'Health Implications'
    elif 'Message' in col_str or 'Cautionary' in col_str:
        new_column_names[col] = 'Message'
    else:
        new_column_names[col] = col  # Keep original if no match

print("Column Renaming Map:")
for old, new in new_column_names.items():
    print(f"  '{old}' -> '{new}'")
print("\n")

aqi_df_cleaned = aqi_df_cleaned.rename(columns=new_column_names)

print("Columns after renaming:")
print(aqi_df_cleaned.columns.tolist())
print("\n")

# 5. Drop the columns 'Health Implications' and 'Message' only if they exist
columns_to_drop = ['Health Implications', 'Message']
columns_to_drop = [col for col in columns_to_drop if col in aqi_df_cleaned.columns]

if columns_to_drop:
    aqi_df_cleaned = aqi_df_cleaned.drop(columns=columns_to_drop)
    print(f"Dropped columns: {columns_to_drop}")
else:
    print("No 'Health Implications' or 'Message' columns found to drop")

print("\nRemaining columns:")
print(aqi_df_cleaned.columns.tolist())
print("\n")

# Check if required columns exist
required_columns = ['AQI Category', 'AQI Range', 'CO', 'C6H6', 'NO2', 'NOx', 'O3']
missing_columns = [col for col in required_columns if col not in aqi_df_cleaned.columns]

if missing_columns:
    print(f"WARNING: Missing required columns: {missing_columns}")
    print("Please check the column renaming map above and adjust accordingly.")
else:
    print("All required columns are present!")

    # 6. Define a helper function parse_concentration_range
    def parse_concentration_range(range_str, pollutant_name):
        range_str = str(range_str).strip()
        lower = 0.0
        upper = np.inf

        if range_str.startswith('>'):
            num_part = re.findall(r'\d+\.?\d*', range_str)[0]
            lower = float(num_part)
        else:
            parts = re.findall(r'\d+\.?\d*', range_str)
            if len(parts) == 1:
                lower = float(parts[0])
                upper = float(parts[0])
            elif len(parts) == 2:
                lower = float(parts[0])
                upper = float(parts[1])
            elif len(parts) > 2:
                lower = float(parts[0])
                upper = float(parts[1])
            else:
                return None

        if pollutant_name in ['C6H6', 'NO2', 'NOx', 'O3']:
            lower /= 1000.0
            if upper != np.inf:
                upper /= 1000.0

        return (lower, upper)

    # 7. Define a helper function parse_aqi_range
    def parse_aqi_range(aqi_range_str):
        aqi_range_str = str(aqi_range_str).strip()
        lower = 0
        upper = np.inf

        if aqi_range_str.startswith('>'):
            lower = int(re.findall(r'\d+', aqi_range_str)[0])
        else:
            parts = aqi_range_str.split('-')
            if len(parts) == 2:
                lower = int(parts[0])
                upper = int(parts[1])
            elif len(parts) == 1 and parts[0].isdigit():
                lower = int(parts[0])
                upper = int(parts[0])
            else:
                return None
        return (lower, upper)

    # 8. Initialize an empty dictionary named aqi_breakpoints
    aqi_breakpoints = {
        'CO': [], 'C6H6': [], 'NO2': [], 'NOx': [], 'O3': []
    }

    # 9. Iterate through the rows and populate aqi_breakpoints
    for index, row in aqi_df_cleaned.iterrows():
        aqi_category = row['AQI Category']
        aqi_range_parsed = parse_aqi_range(row['AQI Range'])

        if aqi_range_parsed is None:
            continue

        for pollutant in ['CO', 'C6H6', 'NO2', 'NOx', 'O3']:
            concentration_range_str = row[pollutant]
            concentration_range_parsed = parse_concentration_range(concentration_range_str, pollutant)

            if concentration_range_parsed is None:
                continue

            aqi_breakpoints[pollutant].append({
                'aqi_category': aqi_category,
                'aqi_range': aqi_range_parsed,
                'concentration_range': concentration_range_parsed
            })

    # 10. Print the aqi_breakpoints dictionary
    print("\n" + "=" * 80)
    print("AQI BREAKPOINTS DICTIONARY:")
    print("=" * 80)
    import json
    print(json.dumps(aqi_breakpoints, indent=4, default=lambda x: str(x) if isinstance(x, (np.ndarray, np.generic)) else x))

    print("\n" + "=" * 80)
    print("CLEANED AQI DATAFRAME:")
    print("=" * 80)
    print(aqi_df_cleaned.head())

**Reasoning**:
The previous attempt to rename columns failed because the `aqi_df_cleaned.columns = aqi_df_cleaned.iloc[0]` step inadvertently assigned `NaN` values as column names due to the structure of the input CSV. The `pd.read_csv` correctly parsed the header of the file, but the first *data row* contained `NaN`s in the 'AQI Category' and 'AQI Range' columns, which was then mistakenly used as the new header. I need to correct this by directly using the column names loaded by `pd.read_csv`, dropping the first data row (index 0), and then renaming the descriptive column names to the shorter pollutant codes (CO, C6H6, etc.).



In [ ]:
import numpy as np
import re # For regular expressions to parse strings

# 1. Create a copy of the aqi_standards_df. It already has the correct initial column names.
aqi_df_cleaned = aqi_standards_df.copy()

# 2. Remove the first data row (index 0) as it contains NaNs for AQI Category and AQI Range.
# The actual data starts from index 1.
aqi_df_cleaned = aqi_df_cleaned[1:].reset_index(drop=True)

# 3. Rename the relevant columns to shorter, more usable names.
# These are the actual column names loaded by pd.read_csv from the header row.
column_rename_map = {
    'AQI Category': 'AQI Category',
    'AQI Range': 'AQI Range',
    'Carbon Monoxide (CO) Concentration (8-hour avg)': 'CO',
    'Benzene (C6H6) Concentration (annual avg)': 'C6H6',
    'Nitrogen Dioxide (NO2) Concentration (?g/m³)': 'NO2', # Note: ?g/m³ is µg/m³
    'Nitrogen Oxides (NOx) Concentration (annual avg)': 'NOx', # Note: ?g/m³ is µg/m³
    'Ozone (O3) Concentration (8-hour avg)': 'O3',
    'Health Implications': 'Health Implications',
    'Message': 'Message'
}
aqi_df_cleaned = aqi_df_cleaned.rename(columns=column_rename_map)

print("Columns after initial renaming:")
print(aqi_df_cleaned.columns.tolist())
print("\n")

# 4. Drop the columns 'Health Implications' and 'Message' as they are not needed for breakpoint lookup.
columns_to_drop = ['Health Implications', 'Message']
# Ensure columns exist before dropping to prevent errors
columns_to_drop = [col for col in columns_to_drop if col in aqi_df_cleaned.columns]

if columns_to_drop:
    aqi_df_cleaned = aqi_df_cleaned.drop(columns=columns_to_drop)
    print(f"Dropped columns: {columns_to_drop}")
else:
    print("No 'Health Implications' or 'Message' columns found to drop.")
print("\n")

print("Final columns in aqi_df_cleaned:")
print(aqi_df_cleaned.columns.tolist())
print("\n")

# 5. Define a helper function parse_concentration_range
def parse_concentration_range(range_str, pollutant_name):
    range_str = str(range_str).strip() # Ensure it's a string and remove whitespace
    lower = 0.0
    upper = np.inf

    # Handle cases where range_str might be 'NaN' or empty
    if range_str == 'nan' or not range_str:
        return None

    # Handle '>' cases first, assuming no '<' cases
    if range_str.startswith('>'):
        num_part = re.findall(r'\d+\.?\d*', range_str)[0]
        lower = float(num_part)
    else:
        # Use a more robust regex to find numerical parts that might be separated by '-' or ' ' or just one number
        parts = re.findall(r'\d+\.?\d*', range_str)
        if len(parts) == 1: # Single value, e.g., '4.4'
            lower = float(parts[0])
            upper = float(parts[0]) # For exact values, though often it's a range
        elif len(parts) >= 2: # 'min-max' or more numbers (take first two as min/max)
            lower = float(parts[0])
            upper = float(parts[1])
        else:
            return None # Handle cases like unexpected format

    # Convert units to mg/m³ where necessary
    # C6H6, NO2, NOx, O3 are in µg/m³ and need to be converted to mg/m³
    if pollutant_name in ['C6H6', 'NO2', 'NOx', 'O3']:
        lower /= 1000.0
        if upper != np.inf:
            upper /= 1000.0

    # For CO, the unit is already mg/m³, no conversion needed.

    return (lower, upper)

# 6. Define a helper function parse_aqi_range
def parse_aqi_range(aqi_range_str):
    aqi_range_str = str(aqi_range_str).strip()
    lower = 0
    upper = np.inf

    if aqi_range_str == 'nan' or not aqi_range_str:
        return None

    if aqi_range_str.startswith('>'):
        lower = int(re.findall(r'\d+', aqi_range_str)[0])
    else:
        parts = aqi_range_str.split('-')
        if len(parts) == 2:
            lower = int(parts[0])
            upper = int(parts[1])
        elif len(parts) == 1 and parts[0].isdigit(): # Single number cases, though not expected from table
            lower = int(parts[0])
            upper = int(parts[0])
        else:
            return None # Handle cases with unexpected format or NaN
    return (lower, upper)

# 7. Initialize an empty dictionary named aqi_breakpoints
aqi_breakpoints = {
    'CO': [], 'C6H6': [], 'NO2': [], 'NOx': [], 'O3': []
}

# 8. Iterate through the rows and populate aqi_breakpoints
for index, row in aqi_df_cleaned.iterrows():
    aqi_category = row['AQI Category']
    aqi_range_parsed = parse_aqi_range(row['AQI Range'])

    if aqi_range_parsed is None:
        continue # Skip rows that couldn't be parsed (e.g. if AQI Range is NaN)

    for pollutant in ['CO', 'C6H6', 'NO2', 'NOx', 'O3']:
        # Check if the pollutant column exists before trying to access it
        if pollutant in aqi_df_cleaned.columns:
            concentration_range_str = row[pollutant]
            concentration_range_parsed = parse_concentration_range(concentration_range_str, pollutant)

            if concentration_range_parsed is None:
                continue # Skip if concentration range couldn't be parsed

            aqi_breakpoints[pollutant].append({
                'aqi_category': aqi_category,
                'aqi_range': aqi_range_parsed,
                'concentration_range': concentration_range_parsed
            })
        else:
            print(f"Warning: Pollutant column '{pollutant}' not found in aqi_df_cleaned. Skipping.")

# 9. Print the aqi_breakpoints dictionary
print("AQI Breakpoints Dictionary:")
import json
print(json.dumps(aqi_breakpoints, indent=4, default=lambda x: str(x) if isinstance(x, (np.ndarray, np.generic)) else x))

print("\nCleaned AQI DataFrame head:")
print(aqi_df_cleaned.head())

**Reasoning**:
The subtask requires extracting specific pollutant data (predicted CO and actual C6H6, NO2, NOx, O3) from existing variables and aligning them with the `y_test_aligned` data for subsequent AQI calculations. This step will gather all necessary pollutant concentrations into a single structured format.



In [ ]:
print("Preparing pollutant data for AQI calculation...")

# The start and end indices for the aligned test data in the original df_clean
# This aligns with the length of ensemble_pred, which is already aligned with the test set.
# The offset comes from SEQUENCE_LENGTH because LSTM predictions start after the initial sequence length.
start_idx_aligned = val_end + SEQUENCE_LENGTH
end_idx_aligned = start_idx_aligned + len(ensemble_pred)

# Extract actual pollutant values from df_clean for the aligned test period
actual_pollutants_aligned = df_clean.iloc[start_idx_aligned:end_idx_aligned].copy()

# Create a dictionary to store all pollutant concentrations (predicted and actual) aligned
pollutant_data = {
    'CO': actual_pollutants_aligned['CO(GT)'].values,
    'C6H6': actual_pollutants_aligned['C6H6(GT)'].values,
    'NO2': actual_pollutants_aligned['NO2(GT)'].values,
    'NOx': actual_pollutants_aligned['NOx(GT)'].values,
    'O3': actual_pollutants_aligned['PT08.S5(O3)'].values
}

# Verify shapes of all pollutant data to ensure alignment
print(f"\nShape of CO: {pollutant_data['CO'].shape}")
print(f"Shape of C6H6: {pollutant_data['C6H6'].shape}")
print(f"Shape of NO2: {pollutant_data['NO2'].shape}")
print(f"Shape of NOx: {pollutant_data['NOx'].shape}")
print(f"Shape of O3: {pollutant_data['O3'].shape}")
print(f"Expected aligned length (ensemble_pred): {len(ensemble_pred)}")

# Display first few values for verification
print("\nFirst 5 aligned pollutant data points:")
for pollutant, values in pollutant_data.items():
    print(f"  {pollutant}: {values[:5]}")

print("Pollutant data prepared successfully!")

In [ ]:
print("Calculating AQI for each pollutant...")

def calculate_individual_aqi(concentration, pollutant_name, aqi_breakpoints):
    """
    Calculates the AQI for a single pollutant concentration using linear interpolation.

    Args:
        concentration (float): The pollutant concentration.
        pollutant_name (str): The name of the pollutant (e.g., 'CO', 'C6H6').
        aqi_breakpoints (dict): Dictionary containing AQI breakpoint ranges.

    Returns:
        float: The calculated AQI value, or np.nan if concentration is invalid.
    """
    if pd.isna(concentration):
        return np.nan

    breakpoints = aqi_breakpoints.get(pollutant_name)
    if not breakpoints:
        return np.nan

    # Sort breakpoints by concentration_range lower bound to ensure correct iteration
    breakpoints = sorted(breakpoints, key=lambda x: x['concentration_range'][0])

    # Handle concentrations below the lowest range
    if concentration <= breakpoints[0]['concentration_range'][0]:
        return breakpoints[0]['aqi_range'][0]

    # Handle concentrations above the highest range if upper limit is infinity
    # This is a general approach for open-ended highest category
    if breakpoints[-1]['concentration_range'][1] == np.inf and concentration > breakpoints[-1]['concentration_range'][0]:
        return breakpoints[-1]['aqi_range'][1]

    for i in range(len(breakpoints) - 1):
        bp_low = breakpoints[i]
        bp_high = breakpoints[i+1]

        conc_low, conc_high = bp_low['concentration_range']
        aqi_low, aqi_high = bp_low['aqi_range']

        # Adjust conc_high to be just before the next category starts if they are contiguous
        # This ensures proper range assignment for boundary values.
        if i < len(breakpoints) - 1:
            next_conc_low = breakpoints[i+1]['concentration_range'][0]
            if conc_high >= next_conc_low: # If ranges overlap or touch
                conc_high = next_conc_low - 1e-9 # A very small epsilon

        if conc_low <= concentration <= conc_high:
            # Linear interpolation formula: I = [(I_high - I_low) / (C_high - C_low)] * (C - C_low) + I_low
            # Ensure C_high - C_low is not zero
            if (conc_high - conc_low) == 0: # Handle single point ranges if they occur
                return float(aqi_low) # Ensure float return
            aqi = ((aqi_high - aqi_low) / (conc_high - conc_low)) * (concentration - conc_low) + aqi_low
            return round(aqi, 2)

    # If concentration falls within the last defined finite range (not caught by loop for highest inf)
    last_bp = breakpoints[-1]
    conc_low, conc_high = last_bp['concentration_range']
    aqi_low, aqi_high = last_bp['aqi_range']
    if conc_high != np.inf and conc_low <= concentration <= conc_high:
        if (conc_high - conc_low) == 0:
             return float(aqi_low) # Ensure float return
        aqi = ((aqi_high - aqi_low) / (conc_high - conc_low)) * (concentration - conc_low) + aqi_low
        return round(aqi, 2)

    return np.nan # Should not be reached for valid concentrations covering all ranges


aqi_values = {}
for pollutant, concentrations in pollutant_data.items():
    # Explicitly specify output type as float to handle NaNs and interpolated values
    vectorized_aqi_calc = np.vectorize(lambda c: calculate_individual_aqi(c, pollutant, aqi_breakpoints), otypes=[np.float64])
    aqis = vectorized_aqi_calc(concentrations)
    aqi_values[pollutant] = aqis

print("AQI values calculated successfully!")
print("\nFirst 5 AQI values for each pollutant:")
for pollutant, aqis in aqi_values.items():
    print(f"  {pollutant}: {aqis[:5]}")

# Verify shapes
print("\nVerifying shapes of calculated AQI values:")
for pollutant, aqis in aqi_values.items():
    print(f"  {pollutant} AQI shape: {aqis.shape}")

In [ ]:
import numpy as np
import pandas as pd # Import pandas for pd.isna

print("Defining the anomaly detection function...")

def detect_anomalies_in_aqi_series(aqi_series, sustained_period=3):
    """
    Detects anomalies in an AQI time series based on predefined rules.

    Args:
        aqi_series (np.ndarray or list): The AQI time series (numerical values).
        sustained_period (int): Number of consecutive hours an AQI must be above 200
                                for a 'Moderate Anomaly' (default: 3).

    Returns:
        np.ndarray: An array of strings indicating the anomaly type for each time step.
    """
    n_steps = len(aqi_series)
    anomaly_types = np.full(n_steps, 'None', dtype='<U10') # Max length 'Emergency'

    # Helper to track consecutive high AQI for 'Moderate Anomaly'
    consecutive_high_aqi_count = 0

    for i in range(n_steps):
        current_aqi = aqi_series[i]
        prev_aqi = aqi_series[i-1] if i > 0 else np.nan # Use NaN for first step

        # Skip anomaly detection if current AQI is NaN
        if pd.isna(current_aqi):
            anomaly_types[i] = 'N/A'
            consecutive_high_aqi_count = 0 # Reset count for NaN values
            continue

        # Reset consecutive count if AQI drops below 200 or is NaN
        if current_aqi <= 200:
            consecutive_high_aqi_count = 0
        else:
            consecutive_high_aqi_count += 1

        # Apply rules in descending order of severity

        # 1. Emergency
        if current_aqi > 400:
            anomaly_types[i] = 'Emergency'
            continue # Move to next time step, this is the most severe

        # 2. Severe Anomaly
        # Check if prev_aqi is not NaN before calculating difference
        if current_aqi > 300 or (not pd.isna(prev_aqi) and (current_aqi - prev_aqi) > 100):
            anomaly_types[i] = 'Severe'
            continue # Move to next time step, this is next most severe

        # 3. Moderate Anomaly
        # Check if current_aqi > 200 AND it has been for sustained_period consecutive hours
        if current_aqi > 200 and consecutive_high_aqi_count >= sustained_period:
            # Need to look back to confirm previous `sustained_period - 1` hours were also > 200
            is_sustained = True
            for j in range(1, sustained_period):
                if i - j < 0 or aqi_series[i-j] <= 200 or pd.isna(aqi_series[i-j]):
                    is_sustained = False
                    break
            if is_sustained:
                anomaly_types[i] = 'Moderate'
                continue # Move to next time step

        # 4. Minor Anomaly
        if not pd.isna(prev_aqi) and (current_aqi - prev_aqi) > 50:
            anomaly_types[i] = 'Minor'

    print("Anomaly detection function defined.")
    return anomaly_types

In [ ]:
print("Applying anomaly detection to individual pollutant AQIs...")

# 1. Initialize an empty dictionary named individual_aqi_anomalies
individual_aqi_anomalies = {}

# 2. Iterate through each pollutant and its corresponding AQI series
for pollutant, aqi_series in aqi_values.items():
    # 3. Call the detect_anomalies_in_aqi_series function
    anomalies = detect_anomalies_in_aqi_series(aqi_series, sustained_period=3)

    # Store the results
    individual_aqi_anomalies[pollutant] = anomalies

print("\nIndividual AQI anomalies detected successfully!")

# 4. After processing all pollutants, iterate through the individual_aqi_anomalies dictionary
print("\nSummary of Individual Pollutant AQI Anomalies:")
print("=" * 60)
for pollutant, anomalies_array in individual_aqi_anomalies.items():
    # 5. Calculate the counts of each unique anomaly type
    unique_anomalies, counts = np.unique(anomalies_array, return_counts=True)
    anomaly_counts = dict(zip(unique_anomalies, counts))

    # Ensure all expected anomaly types are present for consistent printing
    all_anomaly_types = ['None', 'Minor', 'Moderate', 'Severe', 'Emergency', 'N/A'] # Added 'N/A' to cover all possibilities
    for anomaly_type in all_anomaly_types:
        anomaly_counts.setdefault(anomaly_type, 0)

    # 6. Print a formatted summary for each pollutant
    print(f"Pollutant: {pollutant}")
    for anomaly_type in all_anomaly_types:
        print(f"  {anomaly_type:<10}: {anomaly_counts[anomaly_type]:>7}")
    print("-" * 20)

print("Anomaly summary completed!")

In [ ]:
print("Applying anomaly detection to individual pollutant AQIs...")

# 1. Initialize an empty dictionary named individual_aqi_anomalies
individual_aqi_anomalies = {}

# 2. Iterate through each pollutant and its corresponding AQI series
for pollutant, aqi_series in aqi_values.items():
    # 3. Call the detect_anomalies_in_aqi_series function
    anomalies = detect_anomalies_in_aqi_series(aqi_series, sustained_period=3)

    # Store the results
    individual_aqi_anomalies[pollutant] = anomalies

print("\nIndividual AQI anomalies detected successfully!")

# 4. After processing all pollutants, iterate through the individual_aqi_anomalies dictionary
print("\nSummary of Individual Pollutant AQI Anomalies:")
print("=" * 60)
for pollutant, anomalies_array in individual_aqi_anomalies.items():
    # 5. Calculate the counts of each unique anomaly type
    unique_anomalies, counts = np.unique(anomalies_array, return_counts=True)
    anomaly_counts = dict(zip(unique_anomalies, counts))

    # Ensure all expected anomaly types are present for consistent printing
    all_anomaly_types = ['None', 'Minor', 'Moderate', 'Severe', 'Emergency', 'N/A']
    for anomaly_type in all_anomaly_types:
        anomaly_counts.setdefault(anomaly_type, 0)

    # 6. Print a formatted summary for each pollutant
    print(f"Pollutant: {pollutant}")
    for anomaly_type in all_anomaly_types:
        print(f"  {anomaly_type:<10}: {anomaly_counts[anomaly_type]:>7}")
    print("-" * 20)

print("Anomaly summary completed!")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re # For parsing AQI ranges

print("Generating graphs for each pollutant showing AQI and actual concentrations with categories...")

# Define the pollutants to visualize
pollutants_to_visualize = ['CO', 'C6H6', 'NO2', 'NOx', 'O3']

# Get the DateTime index corresponding to the aligned test data
# Ensure start_idx_aligned and end_idx_aligned are defined (from pollutant_data preparation)
time_index_aligned = df_clean['DateTime'].iloc[start_idx_aligned:end_idx_aligned].reset_index(drop=True)

# Define a color palette for AQI categories for better visualization
aqi_colors = {
    'Good': 'green',
    'Satisfactory': 'yellowgreen',
    'Moderate': 'orange',
    'Poor': 'red',
    'Very Poor': 'purple',
    'Severe': 'darkred'
}

fig, axes = plt.subplots(len(pollutants_to_visualize), 1, figsize=(18, 7 * len(pollutants_to_visualize)), sharex=True)
fig.suptitle('Individual Pollutant Concentrations and AQI by Category Over Time', fontsize=20, fontweight='bold', y=1.02)

# Ensure axes is an array even for a single subplot
if len(pollutants_to_visualize) == 1:
    axes = [axes]

# Pre-parse AQI category ranges for plotting
aqi_category_ranges = {}
for _, row in aqi_df_cleaned.iterrows():
    category = row['AQI Category']
    aqi_range_str = str(row['AQI Range']).strip()

    aqi_lower, aqi_upper = (0, np.inf) # Default values
    if '-' in aqi_range_str:
        parts = aqi_range_str.split('-')
        aqi_lower = int(parts[0])
        aqi_upper = int(parts[1])
    elif '>' in aqi_range_str:
        aqi_lower = int(re.findall(r'\d+', aqi_range_str)[0])
        aqi_upper = np.inf # Open-ended upper bound

    aqi_category_ranges[category] = {'lower': aqi_lower, 'upper': aqi_upper}

# Sort categories by their lower bounds for consistent plotting order
sorted_categories = sorted(aqi_category_ranges.items(), key=lambda item: item[1]['lower'])

for i, pollutant in enumerate(pollutants_to_visualize):
    ax1 = axes[i]

    # --- Plot pollutant concentration on left y-axis ---
    sns.lineplot(
        x=time_index_aligned,
        y=pollutant_data[pollutant],
        ax=ax1,
        color='blue',
        label=f'{pollutant} Concentration',
        linewidth=1.5,
        alpha=0.7
    )
    ax1.set_ylabel(f'{pollutant} Concentration', color='blue', fontsize=12, fontweight='bold')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.legend(loc='upper left', bbox_to_anchor=(0.0, 1.0))
    ax1.grid(True, alpha=0.3)

    # --- Create a second y-axis for AQI ---
    ax2 = ax1.twinx()
    sns.lineplot(
        x=time_index_aligned,
        y=aqi_values[pollutant],
        ax=ax2,
        color='red',
        label=f'{pollutant} AQI',
        linewidth=1.5,
        linestyle='--'
    )
    ax2.set_ylabel(f'{pollutant} AQI', color='red', fontsize=12, fontweight='bold')
    ax2.tick_params(axis='y', labelcolor='red')
    ax2.legend(loc='upper left', bbox_to_anchor=(0.0, 0.9))

    # --- Add AQI category thresholds and shaded regions ---
    # Collect legend handles for categories to avoid duplicate entries
    category_legend_handles = []

    for category, bounds in sorted_categories:
        lower = bounds['lower']
        upper = bounds['upper']
        color = aqi_colors.get(category, 'gray')

        # Add shaded region for this AQI category
        if upper != np.inf:
            ax2.axhspan(lower, upper, color=color, alpha=0.1, zorder=0) # Below lines
            # Add horizontal line at upper threshold
            line, = ax2.plot([], [], color=color, linestyle=':', linewidth=1.5, alpha=0.7, label=f'{category} AQI')
            ax2.axhline(y=upper, color=color, linestyle=':', linewidth=1.5, alpha=0.7, zorder=1)
        else:
            # For 'Severe' category (open-ended), shade up to the max visible y-limit
            ax2.axhspan(lower, ax2.get_ylim()[1], color=color, alpha=0.1, zorder=0)
            line, = ax2.plot([], [], color=color, linestyle=':', linewidth=1.5, alpha=0.7, label=f'{category} AQI')
            ax2.axhline(y=lower, color=color, linestyle=':', linewidth=1.5, alpha=0.7, zorder=1)
        category_legend_handles.append(line)

    ax2.set_title(f'{pollutant} Concentration and AQI by Category', fontsize=14, fontweight='bold')
    ax2.grid(False) # Disable grid for second axis to avoid clutter

    # Combine legends for clarity
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax2.legend(h1 + h2, l1 + l2, loc='upper left', bbox_to_anchor=(0.0, 1.0))

    # Set x-axis labels only for the last subplot (or current if only one)
    if i == len(pollutants_to_visualize) - 1:
        ax1.set_xlabel('Date and Time', fontsize=12)
        ax1.tick_params(axis='x', rotation=45)
        for label in ax1.get_xticklabels():
            label.set_ha('right')

plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent suptitle overlap
plt.show()

print("Individual pollutant AQI and concentration visualization with categories completed!")